In [15]:
import struct
import numpy as np

files = ["fashion-mnist_test.csv", "fashion-mnist_train.csv"]
for file in files:
    xs, ys = [], []
    with open(file, "r") as f:
        data = f.read()
    data = data.split("\n")[1:-1]
    for line in data:
        label, *pixels = line.split(",")
        xs.append([float(int(v)) / 255.0 for v in pixels])
        ys.append(int(label))
        
    xs = np.array(xs).astype(np.float32) 
    ys = np.array(ys).astype(np.uint8)
    
    xs = xs.tobytes()
    ys = ys.tobytes()
    
    with open(file.split(".")[0] + "-data.bin", "wb") as f:
        f.write(xs)
    with open(file.split(".")[0] + "-labels.bin", "wb") as f:
        f.write(ys)